<script src="https://use.fontawesome.com/2e576da815.js"></script>

<div style="text-align: left;"><img src="https://www.juliabox.org/assets/img/juliacloudlogo.png" style="margin: 0px 0px 0px 0px; padding-right: 20px;width: 80px; float: left;" title="" alt="" /></div>
<img src="http://dmkpress.com/images/cms/thumbs/a5b0aeaa3fa7d6e58d75710c18673bd7ec6d5f6d/978-5-97060-370-3_270_369__100.jpg" style="margin: 0px 0px 5px 20px; width: 100px; float: right;" title="" alt="" />
Всестороннее введение в новый язык программирования для научно-технических вычислений [Julia](http://julialang.org/) в книге Малколма Шеррингтона, Packt Publishing, июль 2015.

<h1>Осваиваем язык Julia</h1><br />

Совершенствование мастерства в области аналитики и программирования при помощи Julia в целях решения задач комплексной обработки данных
<div style="text-align: left;font-size:8pt;padding-top:10px;">Программный код Julia (v0.4.5) протестирован в Windows 8.1/10 и Linux/Lubuntu 16.4</div>
<div style="text-align: left;"><h1>Глава 8. Базы данных</h1></div>

## Реляционные базы данных
### Сборка и загрузка

<span style="font-family:Consolas, monospace; font-weight:bold">julia<i class="fa fa-terminal" aria-hidden="true"></i></span>

In [ ]:
#! /Users/malcolm/bin/julia
#
# etl.jl
#

# Проверить число аргументов, вывести описание, если не 1 или 2..

nargs = length(ARGS);

if nargs == 0 || nargs > 2 
    println("использование: etl.jl входной_файл [выходной_файл]");
    exit(); 
end

# Первый аргумент - входной файл
# Второй (необязательный) - выходной файл, иначе направить в STDOUT

infile = ARGS[1];

if nargs == 2 
   outfile = ARGS[2];
   try 
      outf = open(outfile,"w");
   catch
      error("Не могу создать выходной файл: ", outfile);
   end
else
   outf = STDOUT;
end

# Функция-однострочник для удвоения одинарных кавычек
escticks(s) = replace(s,"'","''");

# Считать весь файл в матрицу, первая размерность – это число строк
qq = readdlm(infile,'\t');
n = size(qq)[1];

# Будем хранить все категории в словаре
j = 0;
cats = Dict{AbstractString,Int64}();

# Основной цикл загрузки таблицы цитат
for i = 1:n
  cat = qq[i,1];
  if haskey(cats,cat)
    jd = cats[cat];
  else
    j = j + 1; jd = j;
    cats[cat] = jd;
  end
  sql = "insert into quotes values($i,$jd,";  
  if (length(qq[i,2]) > 0)
    sql *= string("'", escticks(qq[i,2]), "',");
  else
    sql *= string("null,");  
  end
  sql *= string("'", escticks(qq[i,3]), "');");
  write(outf,"$sql\n");
end

# Теперь выдадим категории
for cat = keys(cats)
  jd = cats[cat];
  write(outf,"insert into categories values($jd,'$cat');\n");  
end

close(outf);  # если выходной_файл = STDOUT, то не даст никаких результатов 

In [8]:
## Перевод данных из файла с разделителями в файл загрузки в базу данных

home    = homedir()
jp      = "julia_projects"

infile  = joinpath(home,jp,"data","quotes.tsv")
outfile = joinpath(home,jp,"data","loader_ex.sql")
runfile = joinpath(home,jp,"code","etl.jl")

run(`julia $(runfile) $(infile) $(outfile)`)
println("SQL-закрузчик успешно записан на диск.")

SQL-закрузчик успешно записан на диск.


### Нативные интерфейсы

In [1]:
cd(joinpath(homedir(),"julia_projects","data"))

using SQLite

db = SQLite.DB("quotes.db")
res = query(db,"select count(*) from quotes");
res

Data.Table:
1x1 Data.Schema:
 count(*)
    Int64
	Column Data:
	[36]

In [2]:
fieldnames(res)

3-element Array{Symbol,1}:
 :schema
 :data  
 :other 

In [65]:
size(res)

(1,1)

In [66]:
res[1,1].value   

36

In [67]:
res = query(db,"select * from quotes");
size(res) 

(36,4)

In [80]:
nq = size(res)[1]; # Число возвращенных записей
for i = 1:nq
    @printf "%d  " res[i,2].value
end;

1  2  3  2  4  5  1  3  2  1  1  5  5  4  1  1  3  3  2  3  5  1  5  5  5  5  5  5  5  1  2  6  6  6  4  2  

In [82]:
res[21,2].value 

5

In [34]:
sql = "select q.quoname, q.author, c.catname from quotes q ";
sql *= "join categories c on q.cid = c.id limit 5";

res = query(db,sql); 

nq = size(res)[1]; # Число возвращенных записей
for i = 1:nq
    @printf "%s : %s,  %s\n" res[i,3].value res[i,2].value res[i,1].value
end;

Words of Wisdom : Hofstadter's Law,   It always takes longer than you expect, even when you take Hofstadter's Law into account.
Science : Noelie Altito,  The shortest distance between two points is under construction.
Science : Shaw's Principle,  Build a system that even a fool can use, and only a fool will want to use it.
Politics : Adolf Hiltler,  The great mass of the people  will more easily fall victims to a big lie than a small one
Books & Plays : G. B. Shaw,  There is no satisfaction in hanging a man who does not object to it


In [20]:
sql = "select q.quoname from quotes q ";
sql *= " where q.author = 'Oscar Wilde'";

res = query(db,sql);

nq = size(res)[1]; # Число возвращенных записей
for i = 1:nq
    println(res[i,1].value);
end;

The only way to get rid of a temptation is to yield to it.
There is only one thing in the world worse than being talked about, and that is not being talked about
I am not at all cynical, I have merely got experience, which, however, is very much the same thing
To love oneself is the beginning of a lifelong romance
We are all in the gutter, but some of us are looking at the stars
London society is full of women of the very highest birth who have, of their own free choice, remained thirty-five for years


In [ ]:
# res = query(db, "SELECT quoname FROM quotes WHERE author REGEXP 'e(?=Oscar)'") 

### Программный интерфейс ODBC

<i class="fa fa-linux" aria-hidden="true"></i>
<i class="fa fa-apple" aria-hidden="true"></i>

In [ ]:
using ODBC

conn = ODBC.connect("Chinook",usr="malcolm",pwd="mypasswd")
res = ODBC.query("select count(*) from Customers", conn)
println("Number of customers: $(res[:K][1])")

In [ ]:
sql  = "select a.LastName, a.FirstName,";
sql *= " count(b.InvoiceId) as Invs, sum(b.Total) as Amt";
sql *= " from Customer a";
sql *= " join Invoice b on a.CustomerId = b.CustomerId";
sql *= " group by a.LastName having Amt >= 45.00";
sql *= " order by Amt desc;";

res = ODBC.query(sql);

for i in 1:size(res)[1]
  LastName = res[:LastName][i]
  FirstName = res[:FirstName][i]
  Invs = res[:Invs][i]
  Amt = res[:Amt][i]
  @printf "%10s %10s %4d %10.2f\n" LastName FirstName Invs Amt
end

In [ ]:
sql  = "select a.LastName, a.FirstName, d.Name as TrackName";
sql *= " from Customer a";
sql *= " join Invoice b on a.CustomerId = b.CustomerId";
sql *= " join InvoiceLine c on b.InvoiceId = c.InvoiceId";
sql *= " join Track d on c.TrackId = d.TrackId";
sql *= " where a.LastName = 'Cunningham' limit 5;";

res = ODBC.query(sql);

for i in 1:size(res)[1]
  LastName = res[:LastName][i]
  FirstName = res[:FirstName][i]
  TrackName = res[:TrackName][i]
  @printf "%15s %15s %15s\n" LastName FirstName TrackName
end

### Другие методы взаимодействия
#### Драйвер для MySQL

In [ ]:
# Пропускаем обработку аргументов
# Принимаем, что входной файл quotes.tsv
# и используем тестовую базу данных

using DBI
using MySQL

escticks(s) = replace(s,"'","''");
qq = readdlm("quotes.tsv",'\t');
n = size(qq)[1];
conn = connect(MySQL5,"localhost","malcolm","mypasswd","test");

# Можно создать таблицу категорий и таблицы цитат:
sql = "create table categories ";
sql *= "(id integer primary key, ";
sql *= "catname varchar(40) not null");
execute(prepare(conn,sql);
if (errcode(conn) > 0)
  error("Отсутствует возможность построить таблицу категорий")
end;

# Примечание: Похожий код для построения таблицы цитат...
j = 0;
cats = Dict{String,Int64}();

# Главный цикл загрузки таблицы цитат
for i = 1:n
  cat = qq[i,1];
  if haskey(cats,cat)
    jd = cats[cat];
  else
    j = j + 1; jd = j;
    cats[cat] = jd;
  end
  sql = "insert into quotes values($i,$jd,";
  if (length(qq[i,2]) > 0)
    sql *= string("'", escticks(qq[i,2]), "',");
  else
    sql *= string("null,");
  end
  sql *= string("'", escticks(qq[i,3]), "');");
  stmt = prepare(conn,sql); execute(stmt);
  if errcode(conn) > 0 error(errstring(conn); end;
end

# Теперь вывести категории
for cat = keys(cats)
  jd = cats[cat];
  sql = "insert into categories values($jd,'$cat')";
  stmt = prepare(conn,sql); execute(stmt);
  if errcode(conn) > 0 error(errstring(conn); end;
end;
disconnect(conn);

#### Драйвер для PostgreSQL

createdb Chinook
psql –d Chinook -a –f Chinook_PostgresSql.sql

In [ ]:
using DBI, PostgreSQL

connect(Postgres) do conn
  stmt = prepare(conn,"SELECT * FROM \"MediaType\"");
  res = execute(stmt);
  for row in res
    @printf "ID: %2d, Media: %s\n" row[1] row[2]
  end
  finish(stmt);
end

### Пакет PyCall

In [ ]:
using PyCall

@pyimport mysql.connector as mc

cnx = mc.connect (user="malcolm", password="mypasswd");
csr = cnx[:cursor]()
query = "SELECT * FROM Chinook.Genre"
csr[:execute](query)

for vals in csr
  id = vals[1]
  genre = vals[2]
  @printf "ID: %2d, %s\n" id genre
end

csr[:close]()
cnx[:close]()

### Стандарт взаимодействия JDBC

In [ ]:
using JavaCall

jsd = @jimport java.sql.DriverManager;
dbURL = "jdbc:derby:Books1";
conn = nothing;
try
  db = jcall(jsd,"getConnection",JavaObject,(JString,),dbURL);
  jsp = @jimport java.sql.PreparedStatement;
  jsr = @jimport java.sql.ResultSet;
  sql = "select count(*) as K from quotes";
  stmt = jcall(db,"prepareStatement",isp,(JString,),sql);
  res = jcall(stmt,"executeQuery",jsr,());
  k = jcall(res,"getString",JString,(JString,),"K");
catch e
  println(e);
finally
  if conn != nothing
    jcall(conn,"close",Void,());
  end;
end;
println("\nЧисло цитат в базе данных: $k);

## Хранилища данных NoSQL
### Системы ключ-значение 

In [ ]:
using Redis;

conn = RedisConnection()
set (conn,"Hello","World");
keys(conn, "*")

In [ ]:
println(get(conn, "Hello"));
disconnect(conn);

In [ ]:
using Quandl, DataFrames, Redis, Winston

qapi_key = "aaaaaaaaaaaaaaaaaaaa" # Your Quandl key

set_auth_token(qapi_key)

qdf1 = quandl("WIKI/AAPL", format="DataFrame", order="asc");
aapl = convert(Array, qdf1[:Close]);
scf1 = 1.0/aapl[1]; aapl = scf1 .* aapl;

qdf2 = quandl("WIKI/MSFT", format="DataFrame", order="asc");
msft = convert(Array, qdf2[:Close]);
scf2 = 1.0/msft[1]; msft = scf2 .* msft;

n = [1:length(aapl)];
conn = RedisConnection()

for i = 1:n
  rpush(conn,'APPL~Close',aapl[i])
  rpush(conn,'MSFT~Close',msft[i])
end

t = [1:n]; # Просто отобразить на графике данные, не даты
aapl-data = float32(lrange(conn,"AAPL~Close",0,-1);
msft-data = float32(lrange(conn,"MSFT~Close",0,-1);

plot(t, aapl-data,"b--", t, msft-data, "r.");

### Документно-ориентированные хранилища данных 

In [ ]:
using Mongo, LibBSON

client = MongoClient(); # По умолчанию localhost порт 27017

mc = MongoCollection(client,"test","quotes");
bob = BSONObject({"author" => "Albert Einstein"});

println("Quotes by Albert Einstein numbers ", count(mc, bob);

In [ ]:
cur = find(mc,bob);  # => MongoCursor(Ptr{Void} @0x00007fadb1806000)
for obj in cur
  println(obj["category"],": ",obj["quote"])
end

In [ ]:
bin = BSONObject({"author" => "Albert Einstein","category" => "Test", "quote" => "I don't believe that God plays dice."});
bup = BSONObject({"author" => "Albert Einstein","category" => "Test", "quote" => "A person who never made a mistake, never
made anything."});
brm = BSONObject({"author" => "Albert Einstein","category" => "Test"}); 

In [ ]:
insert(mc,bin) # => BSONOID(54e1201c52f2a1f4f8289b61)
count(mc,bob)  # => 3
update(mc,bin,bup) # => true
count(mc,bob)  # => 3
delete(mc,brm) # => true
count(mc,bob)  # => 2

### Взаимодействие с RESTful

In [11]:
# rts = run(`wget –q –O- http://amisllp.com /now.php`)
rts = run(`curl –X GET http://amisllp.com/now.php`)

#### Данные в формате JSON

In [1]:
using HTTPClient.HTTPC

url = "http://httpbin.org/ip"
uu = HTTPC.get(url);
fieldnames(uu)'

1x5 Array{Symbol,2}:
 :body  :headers  :http_code  :total_time  :bytes_recd

In [2]:
using JSON

ubody = bytestring(uu.body.data);
ss = JSON.parse(ubody);
myip = ss["origin"]

@printf "My IP address is %s\n" myip

My IP address is 85.15.89.45


In [4]:
#
# rest_books.jl
#

using Requests
#import Requests: get, post, put, delete, options
using JSON

api = "https://www.googleapis.com/books/v1/volumes"
url = api * "?q=isbn:1408855895";

uu = Requests.get(url);

json = JSON.parse(bytestring(uu.data));

volumeInfo = json["items"][1]["volumeInfo"];
title = volumeInfo["title"];
author = volumeInfo["authors"][1];
publisher = volumeInfo["publisher"];
pubdate = volumeInfo["publishedDate"];
ppcount = volumeInfo["pageCount"];

@printf "%s, автор: %s\nОпубликовано: %s (%s), кво страниц: %d" title author publisher pubdate ppcount

Harry Potter and the Philosopher's Stone, автор: J. K. Rowling
Опубликовано: Bloomsbury Publishing (2014-09-01), кво страниц: 224

### Интернет-СУБД

In [12]:
using Requests, JSON

url = "http://localhost:5984"
uu = Requests.get(url);
json = JSON.parse(bytestring(uu.data))

Dict{AbstractString,Any} with 4 entries:
  "uuid"    => "07fb3f7ef7db5835f07828f2c233474b"
  "couchdb" => "Welcome"
  "version" => "1.6.1"
  "vendor"  => Dict{AbstractString,Any}("name"=>"The Apache Software Foundation…

In [ ]:
# Создать новую базу данных

# run(`curl -X PUT http://localhost:5984/quotes_ru`)   #/цитаты ?

url = "http://localhost:5984/quotes"
Requests.put(url);

In [13]:
#run(`curl http://localhost:5984/_all_dbs`)

url = "http://localhost:5984/_all_dbs"
uu = Requests.get(url);
json = JSON.parse(bytestring(uu.data))

3-element Array{Any,1}:
 "_replicator"
 "_users"     
 "quotes"     

In [19]:
# curl -H 'Content-Type: application/json' -X POST http://127.0.0.1:5984/quotes -d '{category:"...",author:"...", "quote":"..."}'

url = "http://localhost:5984/quotes"
json = Dict("category" => "Computing", 
            "author" => "Scott's Law", 
            "quote" => "Adding manpower to a late software project makes it later") 
uu = Requests.post(url, json = json)

Response(201 Created, 11 headers, 95 bytes in body)

In [20]:
cd(joinpath(homedir(),"julia_projects","data"))
include("quotes_arr.jl")
 
using Requests, JSON

cdb = "http://localhost:5984";
qc = cdb * "/quotes";

for json in quotes_arr
  Requests.post("$qc", json=json)
end

println("Данные успешно внесены.")

In [14]:
cdb = "http://localhost:5984"
qc_all = cdb * "/quotes/_all_docs"

res = bytestring(Requests.get(qc_all).data);
json = JSON.parse(res) 

Dict{AbstractString,Any} with 3 entries:
  "rows"       => Any[Dict{AbstractString,Any}("key"=>"a1b231f6d4193e03702d181b…
  "offset"     => 0
  "total_rows" => 2

In [15]:
json["total_rows"]    

2

In [16]:
json["rows"][2]      # Выделить конкретную запись

Dict{AbstractString,Any} with 3 entries:
  "key"   => "a1b231f6d4193e03702d181be90058d9"
  "id"    => "a1b231f6d4193e03702d181be90058d9"
  "value" => Dict{AbstractString,Any}("rev"=>"1-c39a3f858486377c80483304e012ce4…

In [18]:
qc = cdb * "/quotes";

key = json["rows"][2]["key"];
rev = json["rows"][2]["value"]["rev"];

JSON.parse(bytestring(Requests.get("$qc/$key").data))

Dict{AbstractString,Any} with 5 entries:
  "quote"    => "I can resist everything but temptation"
  "_rev"     => "1-c39a3f858486377c80483304e012ce47"
  "author"   => "Oscar Wilde"
  "_id"      => "a1b231f6d4193e03702d181be90058d9"
  "category" => "Books & Plays"

In [ ]:
Requests.delete("$qc/$key?rev=$rev")

In [ ]:
json = Dict("category" => "Books & Plays", "author" => "Oscar Wilde", "quote" => "I can resist everything but temptation")

Requests.delete("$qc/$key?rev=$rev")
Requests.post("$qc", json=json)

### Графовые системы хранения данных

In [ ]:
using HTTPClient.HTTPC, JSON;

cypher = "http://localhost:7474/db/data/cypher";
ropts = RequestOptions(content_type="application/json");
match = "START r=node(*) RETURN count(r)"
rts = HTTPC.post(cypher,"{\"query\":\"$match\"}",ropts);
json = JSON.parse(string(rts));
json.body.columns; # => [ "count(r)" ];
json.body.data; # => [[ 171 ]];

# И используя MATCH = "START r=rel(*) RETUrn count(r)"
json.body.columns; # => [ "count(r)" ];
json.body.data; # => [[ 253 ]];

In [ ]:
match = "MATCH (people:Person) RETURN people.name LIMIT 3";
json.body.columns; # => [ "people.name" ]
json.body.data
[["Keanu-Reeves"],["Carrie-Ann Moss"],["Laurence Fishburne"]]

match = "MATCH ((tom:Person {name:"Tom Hanks"})-
  [:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors)
  RETURN coActors.name LIMIT 3";

json.body.columns; # => [ "coActors.name" ]
json.body.data'
[["Meg Ryan"],["Greg Kinnear"],["Parker Posey"]] 